# BayesianSearchCV

1. Apa itu hyperparameter tuning?
Hyperparameter = parameter yang tidak dipelajari langsung dari data, tapi ditentukan sebelum training.
Hyperparameter tuning = proses mencari kombinasi hyperparameter terbaik supaya model punya performa optimal (akurasi tinggi, error rendah, dll).

2. Jelaskan metode BayesSearchCV!
BayesSearchCV adalah implementasi hyperparameter tuning berbasis Bayesian Optimization dari library scikit-optimize (skopt).

3. Bagaimana cara kerja BayesSearchCV?
  1. Mulai dengan sampling awal: memilih beberapa kombinasi hyperparameter secara acak.
  2. Bangun model probabilistik (surrogate model), biasanya Gaussian Process, untuk memprediksi kualitas dari kombinasi hyperparameter.
  3. Acquisition function: menentukan kombinasi hyperparameter baru yang kemungkinan besar menghasilkan perbaikan performa.
  4. Evaluasi model dengan kombinasi tersebut.
  5. Update surrogate model dengan hasil baru.
  6. Iterasi langkah 3–5 sampai batas iterasi/ waktu tercapai.

4. Apa kelebihan BayesSearchCV dibandingkan metode hyperparameter tuning lainnya?
  1. Lebih efisien dari GridSearchCV
     GridSearch: coba semua kombinasi → boros waktu.
     BayesSearchCV: coba kombinasi yang paling menjanjikan → hemat waktu.
  2. Lebih efektif dari RandomizedSearchCV
     Random: sampling murni acak, bisa buang waktu di area buruk.
     Bayes: belajar dari hasil sebelumnya, makin lama makin fokus ke area “bagus”.
  3. Terintegrasi dengan scikit-learn
     Sama seperti GridSearchCV → gampang dipakai dengan pipeline, cross-validation, dsb.
  4. Cocok untuk masalah dengan hyperparameter banyak & mahal dihitung
     Misalnya training model besar (SVM, XGBoost, RandomForest).



In [11]:
# import library
#!pip install scikit-optimize # install jika belum pernah install #

# import library setelah install
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV


In [12]:
# Load Dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# BayesSearchCV dari scikit-optimize
from skopt import BayesSearchCV

# 1. Load dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.75, random_state=0
)

# 2. Definisikan model
rf = RandomForestClassifier(random_state=0)

# 3. Tentukan search space hyperparameter
param_space = {
    'n_estimators': (10, 200),
    'max_depth': (1, 20),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10)
}

# 4. Inisialisasi BayesSearchCV
opt = BayesSearchCV(
    rf,
    search_spaces=param_space,
    n_iter=30,
    cv=3,
    scoring='accuracy',
    random_state=0,
    n_jobs=-1
)

# 5. Fit ke data training
opt.fit(X_train, y_train)

# 6. Hasil terbaik
print("Best parameters:", opt.best_params_)
print("Best CV score:", opt.best_score_)

# 7. Evaluasi di test set
y_pred = opt.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Best parameters: OrderedDict({'max_depth': 11, 'min_samples_leaf': 7, 'min_samples_split': 13, 'n_estimators': 133})
Best CV score: 0.9556661925082978
Test Accuracy: 0.9736842105263158


In [14]:
# Definisikan Model dan parameter yang akan dioptimasi (Ruang Hyperparameter)
# Definisikan Model
rf = RandomForestClassifier(random_state=0)

# Ruang Hyperparameter yang mau dioptimasi
param_space = {
    'n_estimators': (10, 200),
    'max_depth': (1, 20),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10)
}

# Fungsi Optimisasi Menggunakan BayesianSearchCV

In [16]:
# Inisialisasi BayesSearchCV
opt = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=30,
    cv=3,
    scoring='accuracy',
    random_state=0,
    n_jobs=-1
)

In [17]:
# Jalankan optimisasi
# Jalankan optimisasi di data training
opt.fit(X_train, y_train)

# Tampilkan hasil terbaik
print("Best Parameters:", opt.best_params_)
print("Best CV Score:", opt.best_score_)

Best Parameters: OrderedDict({'max_depth': 11, 'min_samples_leaf': 7, 'min_samples_split': 13, 'n_estimators': 133})
Best CV Score: 0.9556661925082978


In [18]:
# Evaluasi Model
from sklearn.metrics import accuracy_score, classification_report

# Prediksi data test dengan model terbaik
y_pred = opt.predict(X_test)

# Evaluasi akurasi
print("Test Accuracy:", accuracy_score(y_test, y_pred))

# Laporan lebih detail
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 0.9736842105263158

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

